In [ ]:
import os
import csv
csv.field_size_limit(100000000)

def load_scripts(directory):
    f = open(directory,'r', encoding='utf-8')
    rdr = csv.reader(f)

    script_list = []
    title_list = []

    # make scriptlist and titlelist
    for idx, line in enumerate(rdr):
        #print(idx)
        #if len(line) <= 2:
         #   continue
        script_list.append(line[2])
        title_list.append(line[1])
    return script_list, title_list

In [ ]:
import csv
import re
import os
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

class BERTModel:
    def __init__(self, script_list, title_list):
        self.script_list = script_list
        self.title_list = title_list
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.script_embeddings = self.generate_embeddings(self.script_list)

    def preprocess_text(self, text):
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # 특수 문자와 숫자 제거
        text = text.lower()  # 소문자로 변환
        text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
        return text

    def bert_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = self.model(**inputs)
        sentence_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
        return sentence_embedding

    def generate_embeddings(self, scripts):
        return [self.bert_embedding(script) for script in scripts]

    def find_similar_movies(self, query, top_n=300): ###### 임의로
        query_embedding = self.bert_embedding(query)
        similarities = [cosine_similarity(query_embedding, script_embedding).flatten()[0] for script_embedding in self.script_embeddings]
        top_indices = np.argsort(similarities)[-top_n:][::-1]
        similar_movies = [(self.title_list[i], similarities[i]) for i in top_indices]
        return similar_movies

# 1. 관우님이 전처리

## 칼럼 안들어가게 항상 1: 해서 슬라이싱해야됨

In [ ]:
script_list[1:], title_list[1:] = load_scripts('/content/scripts.csv')
bert_model = BERTModel(script_list, title_list)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# 다시 모델 불러오지 않기 위해 일단 클래스 사용안함

In [ ]:
def preprocess_text(text):
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # 특수 문자와 숫자 제거
        text = text.lower()  # 소문자로 변환
        text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
        return text

def bert_embedding(text):
        inputs = bert_model.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = bert_model.model(**inputs)
        sentence_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
        return sentence_embedding

def generate_embeddings(scripts):
        return [bert_model.bert_embedding(script) for script in scripts]

def find_similar_movies(query, top_n=300): ###### 임의로
        query = bert_model.preprocess_text(query)  # Ensure query is preprocessed
        query_embedding = bert_model.bert_embedding(query)
        print(query_embedding)
        similarities = [cosine_similarity(query_embedding, script_embedding).flatten()[0] for script_embedding in bert_model.script_embeddings]
        top_indices = np.argsort(similarities)[-top_n:][::-1]
        similar_movies = [(bert_model.title_list[i], similarities[i]) for i in top_indices]
        return similar_movies

# query ㄱㄱ

In [ ]:
similar_movies_love = find_similar_movies('love')
similar_movies_romance = find_similar_movies('romance')
similar_movies_familiy = find_similar_movies('familiy')

In [ ]:
similar_movies_love == similar_movies_familiy

False

In [ ]:
similar_movies_love == similar_movies_romance

False

In [ ]:
similar_movies_romance == similar_movies_familiy

False

# 다들 evaluation 어떻게 하신건지?

In [ ]:
#f = open('/content/movie_details_no_scripted.csv', 'r', encoding='utf-8')
#애초에 인덱스값이 달라서 할 수가 없음
rdr = csv.reader(f)

genrelist = []
for line in rdr:
    if len(line) <= 4:
        genrelist.append("blank")
    else:
        genrelist.append(line[4])
f.close()

# Evaluation
tp = 0
fp = 0
fn = 0

for title, _ in similar_movies:
    if "Romance" in genrelist[index]:
        tp += 1
    else:
        fp += 1

rel = genrelist.count("Romance")
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall) # f1 score도 구현하기

Precision: 0.08571428571428572
Recall: 0


# evaluation

In [ ]:
# Open the movie details CSV file with utf-8 encoding
f = open('/content/evaluation.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
# Initialize genre lists
romance_list = []
scifi_list = []
family_list = []

for line in rdr:
    if len(line) > 1:
        romance_list.append(line[0])
    if len(line) > 2:
        scifi_list.append(line[1])
    if len(line) > 3:
        family_list.append(line[2])
f.close()

In [ ]:
import csv

def load_genre_lists(csv_file):
    # Initialize genre lists
    romance_list = []
    scifi_list = []
    family_list = []

    # Open the movie details CSV file with utf-8 encoding
    with open(csv_file, 'r', encoding='utf-8') as f:
        rdr = csv.reader(f)
        next(rdr)  # Skip the header row

        # Read the CSV file and store the movie titles in the respective lists
        for line in rdr:
            if len(line) > 0 and line[0].strip():
                romance_list.append(line[0].strip())
            if len(line) > 1 and line[1].strip():
                scifi_list.append(line[1].strip())
            if len(line) > 2 and line[2].strip():
                family_list.append(line[2].strip())

    return romance_list, scifi_list, family_list

# Example usage
csv_file = '/content/evaluation.csv'
romance_list, scifi_list, family_list = load_genre_lists(csv_file)

print("Romance list:", romance_list)
print("Sci-fi list:", scifi_list)
print("Family list:", family_list)


Romance list: ['10 Things I Hate About You', '17 Again', '500 Days of Summer', 'Adjustment Bureau, The', 'Adventures of Buckaroo Banzai Across the Eighth Dimension, The', 'Airplane', 'Almost Famous', 'American President, The', 'American Werewolf in London', 'Amour', 'Angel Eyes', 'Annie Hall', 'Artist, The', 'As Good As It Gets', 'Autumn in New York', 'Back-up Plan, The', 'Barry Lyndon', 'Battle of Shaker Heights, The', 'Beginners', 'Belle', 'Benny & Joon', 'Big Blue, The', 'Big Sick, The', 'Birdman', 'Birds, The', 'Birthday Girl', 'Blast from the Past, The', 'Blue Valentine', 'Bodies, Rest & Motion', 'Body of Evidence', 'Bodyguard', 'Bonnie and Clyde', 'Bounty Hunter, The', 'Broadcast News', 'Broken Embraces', 'Brothers Bloom, The', 'Bruce Almighty', 'Bull Durham', 'Burlesque', 'Burning Annie', 'Casablanca', 'Celeste & Jesse Forever', 'Charade', 'Chasing Amy', 'Cider House Rules, The', 'Cinema Paradiso', 'Clueless', 'Cold Mountain', 'Color of Night', 'Cooler, The', 'Crash', 'Crazy, St

In [ ]:
family_list

['Addams Family, The',
 'Aladdin',
 'Anastasia',
 'Back to the Future',
 'Back to the Future II & III',
 'Beauty and the Beast',
 'Big',
 'Chronicles of Narnia: The Lion, the Witch and the Wardrobe',
 'Coraline',
 'Duck Soup',
 'E.T.',
 'FairyTale: A True Story',
 'Fantastic Mr Fox',
 'Field of Dreams',
 'Finding Nemo',
 'Flintstones, The',
 'Frankenweenie',
 'Happy Feet',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Half-Blood Prince',
 'Harry Potter and the Prisoner of Azkaban',
 "Harry Potter and the Sorcerer's Stone",
 'Incredibles, The',
 "It's a Wonderful Life",
 'Kung Fu Panda',
 'Labyrinth',
 'Lion King, The',
 'Little Mermaid, The',
 'Marley & Me',
 'Mary Poppins',
 'Megamind',
 'Mighty Joe Young',
 'Mighty Morphin Power Rangers: The Movie',
 'Mulan',
 'My Girl',
 'Neverending Story, The',
 'New York Minute',
 'Newsies',
 'Nightmare Before Christmas, The',
 'Pacifier, The',
 'Pokemon: Mewtwo Returns',
 'Princess Bride, The',
 'Rescuers Down Under, The',
 

In [ ]:
# Evaluation
tp = 0
fp = 0
fn = 0

for title, _ in similar_movies_love:
    if title in romance_list:
        print(title)
        tp += 1
    else:
        fp += 1

rel = len(romance_list)
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Airplane
Adventures of Buckaroo Banzai Across the Eighth Dimension, The
17 Again
Adjustment Bureau, The
10 Things I Hate About You
500 Days of Summer
Precision: 0.13636363636363635
Recall: 0.02843601895734597


In [ ]:
# Evaluation
tp = 0
fp = 0
fn = 0

for title, _ in similar_movies_romance:
    if title in romance_list:
        print(title)
        tp += 1
    else:
        fp += 1

rel = len(romance_list)
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Airplane
Adventures of Buckaroo Banzai Across the Eighth Dimension, The
17 Again
Adjustment Bureau, The
10 Things I Hate About You
500 Days of Summer
Precision: 0.13636363636363635
Recall: 0.02843601895734597


# 관우님 결과 >> 유사도 점수만 엄청 조금 차이나고 똑같은 순위로 똑같은 영화가 계속 결과로 나왔음

# 이번엔 내가 전처리한걸로 해보기(1087)

In [ ]:
import os
import csv
csv.field_size_limit(100000000)

def load_scripts(directory):
    f = open(directory,'r', encoding='utf-8')
    rdr = csv.reader(f)

    script_list = []
    title_list = []

    # make scriptlist and titlelist
    for idx, line in enumerate(rdr):
        #print(idx)
        #if len(line) <= 2:
         #   continue
        script_list.append(line[1])
        title_list.append(line[0])
    return script_list, title_list

In [ ]:
import csv
import re
import os
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

class BERTModel:
    def __init__(self, script_list, title_list):
        self.script_list = script_list
        self.title_list = title_list
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')
        self.script_embeddings = self.generate_embeddings(self.script_list)

    def preprocess_text(self, text):
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # 특수 문자와 숫자 제거
        text = text.lower()  # 소문자로 변환
        text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
        return text

    def bert_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = self.model(**inputs)
        sentence_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
        return sentence_embedding

    def generate_embeddings(self, scripts):
        return [self.bert_embedding(script) for script in scripts]

    def find_similar_movies(self, query, top_n=300): ###### 임의로
        query_embedding = self.bert_embedding(query)
        similarities = [cosine_similarity(query_embedding, script_embedding).flatten()[0] for script_embedding in self.script_embeddings]
        top_indices = np.argsort(similarities)[-top_n:][::-1]
        similar_movies = [(self.title_list[i], similarities[i]) for i in top_indices]
        return similar_movies

In [ ]:
script_list, title_list = load_scripts('/content/uiji_preprocessd_scripts_2.csv')
bert_model_uiji = BERTModel(script_list, title_list)

In [ ]:
def preprocess_text(text):
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # 특수 문자와 숫자 제거
        text = text.lower()  # 소문자로 변환
        text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
        return text

def bert_embedding(text):
        inputs = bert_model_uiji.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = bert_model_uiji.model(**inputs)
        sentence_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
        return sentence_embedding

def generate_embeddings(scripts):
        return [bert_model_uiji.bert_embedding(script) for script in scripts]

def find_similar_movies(query, top_n=300): ###### 임의로
        query = bert_model_uiji.preprocess_text(query)  # Ensure query is preprocessed
        query_embedding = bert_model_uiji.bert_embedding(query)
        #print(query_embedding)
        similarities = [cosine_similarity(query_embedding, script_embedding).flatten()[0] for script_embedding in bert_model_uiji.script_embeddings]
        top_indices = np.argsort(similarities)[-top_n:][::-1]
        similar_movies = [(bert_model_uiji.title_list[i], similarities[i]) for i in top_indices]
        return similar_movies

In [ ]:
similar_movies_love = find_similar_movies('love')
similar_movies_romance = find_similar_movies('romance')
similar_movies_familiy = find_similar_movies('familiy')

In [ ]:
similar_movies_love = similar_movies_love[1:] # 칼럼빼기
len(similar_movies_love)

299

# 내가 전처리 한거에 포함안되는 건 evaluation set에서도 빼야됨

In [ ]:
import csv

def load_genre_lists(csv_file, title_list):
    # Initialize genre lists
    romance_list = []
    scifi_list = []
    family_list = []

    # Open the movie details CSV file with utf-8 encoding
    with open(csv_file, 'r', encoding='utf-8') as f:
        rdr = csv.reader(f)
        next(rdr)  # Skip the header row

        # Read the CSV file and store the movie titles in the respective lists
        for line in rdr:
            if len(line) > 0 and line[0].strip() and line[0].strip() in title_list:# 데이터셋에 포함안된애들만
                romance_list.append(line[0].strip())
            if len(line) > 1 and line[1].strip() and line[1].strip() in title_list:
                scifi_list.append(line[1].strip())
            if len(line) > 2 and line[2].strip() and line[2].strip() in title_list:
                family_list.append(line[2].strip())

    return romance_list, scifi_list, family_list

csv_file = '/content/evaluation.csv'
romance_list, scifi_list, family_list = load_genre_lists(csv_file, title_list)

print("Romance list:", romance_list)
print("Sci-fi list:", scifi_list)
print("Family list:", family_list)


Romance list: ['10 Things I Hate About You', '17 Again', '500 Days of Summer', 'Adjustment Bureau, The', 'Adventures of Buckaroo Banzai Across the Eighth Dimension, The', 'Airplane', 'Almost Famous', 'American President, The', 'American Werewolf in London', 'Amour', 'Angel Eyes', 'Annie Hall', 'Artist, The', 'As Good As It Gets', 'Autumn in New York', 'Back-up Plan, The', 'Barry Lyndon', 'Battle of Shaker Heights, The', 'Beginners', 'Belle', 'Big Blue, The', 'Big Sick, The', 'Birdman', 'Birds, The', 'Birthday Girl', 'Blue Valentine', 'Body of Evidence', 'Bodyguard', 'Bonnie and Clyde', 'Bounty Hunter, The', 'Broadcast News', 'Broken Embraces', 'Brothers Bloom, The', 'Bruce Almighty', 'Bull Durham', 'Burlesque', 'Burning Annie', 'Charade', 'Chasing Amy', 'Cider House Rules, The', 'Cinema Paradiso', 'Cold Mountain', 'Color of Night', 'Cooler, The', 'Crash', 'Crazy, Stupid, Love', 'Crazylove', 'Crouching Tiger, Hidden Dragon', 'Crow, The', 'Cruel Intentions', 'Crying Game', 'Curious Case 

In [ ]:
len(romance_list)

In [ ]:
# Evaluation
tp = 0
fp = 0
fn = 0

for title, _ in similar_movies_love:
    if title in romance_list:
        print(title)
        tp += 1
    else:
        fp += 1

rel = len(romance_list)
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Punch-Drunk Love
Saint, The
Almost Famous
Meet Joe Black
Broadcast News
True Romance
Eternal Sunshine of the Spotless Mind
Ninotchka
High Fidelity
Wild At Heart
Fabulous Baker Boys, The
Crazy, Stupid, Love
Shakespeare in Love
Last of the Mohicans, The
Twilight
Burlesque
Crash
American President, The
Jane Eyre
From Here to Eternity
It's Complicated
Silver Linings Playbook
Juno
American Werewolf in London
Spanglish
Airplane
Pride and Prejudice
Bruce Almighty
Sessions, The
Larry Crowne
Midnight in Paris
Twilight: New Moon
Rocky
It Happened One Night
It Happened One Night
Monte Carlo
Crazylove
Her
Les Miserables
Zerophilia
Three Musketeers, The
His Girl Friday
Next Three Days, The
Yes Man
Cooler, The
Princess Bride, The
As Good As It Gets
Terminator
Up in the Air
Shipping News, The
Reader, The
Things My Father Never Taught Me, The
Tin Cup
Curious Case of Benjamin Button, The
Natural Born Killers
Mrs. Brown
Precision: 0.18729096989966554
Recall: 0.2994652406417112


In [ ]:
# Evaluation
tp = 0
fp = 0
fn = 0

for title, _ in similar_movies_romance:
    if title in romance_list:
        print(title)
        tp += 1
    else:
        fp += 1

rel = len(romance_list)
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Punch-Drunk Love
Saint, The
Almost Famous
True Romance
Meet Joe Black
Broadcast News
Ninotchka
Eternal Sunshine of the Spotless Mind
High Fidelity
Burlesque
Wild At Heart
Last of the Mohicans, The
Shakespeare in Love
Fabulous Baker Boys, The
Crazy, Stupid, Love
Crash
Twilight
American Werewolf in London
From Here to Eternity
Juno
Spanglish
American President, The
Airplane
Silver Linings Playbook
It's Complicated
Jane Eyre
Pride and Prejudice
Bruce Almighty
Larry Crowne
Rocky
Crazylove
Sessions, The
Midnight in Paris
Monte Carlo
Twilight: New Moon
Cooler, The
It Happened One Night
It Happened One Night
Three Musketeers, The
Her
His Girl Friday
Next Three Days, The
Les Miserables
Princess Bride, The
Up in the Air
Mrs. Brown
Terminator
Yes Man
Natural Born Killers
Zerophilia
Intolerable Cruelty
Ghost
Things My Father Never Taught Me, The
Titanic
Shipping News, The
Reader, The
Platinum Blonde
Precision: 0.19063545150501673
Recall: 0.3048128342245989


In [ ]:
similar_movies_romance = similar_movies_romance[1:]
len(similar_movies_romance)

298

In [ ]:
tp = 0
fp = 0
fn = 0

for title, _ in similar_movies_romance:
    if title in romance_list:
        print(title)
        tp += 1
    else:
        fp += 1

rel = len(romance_list)
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Saint, The
Almost Famous
True Romance
Meet Joe Black
Broadcast News
Ninotchka
Eternal Sunshine of the Spotless Mind
High Fidelity
Burlesque
Wild At Heart
Last of the Mohicans, The
Shakespeare in Love
Fabulous Baker Boys, The
Crazy, Stupid, Love
Crash
Twilight
American Werewolf in London
From Here to Eternity
Juno
Spanglish
American President, The
Airplane
Silver Linings Playbook
It's Complicated
Jane Eyre
Pride and Prejudice
Bruce Almighty
Larry Crowne
Rocky
Crazylove
Sessions, The
Midnight in Paris
Monte Carlo
Twilight: New Moon
Cooler, The
It Happened One Night
It Happened One Night
Three Musketeers, The
Her
His Girl Friday
Next Three Days, The
Les Miserables
Princess Bride, The
Up in the Air
Mrs. Brown
Terminator
Yes Man
Natural Born Killers
Zerophilia
Intolerable Cruelty
Ghost
Things My Father Never Taught Me, The
Titanic
Shipping News, The
Reader, The
Platinum Blonde
Precision: 0.18791946308724833
Recall: 0.2994652406417112


In [ ]:
similar_movies_familiy = similar_movies_familiy[1:]
tp = 0
fp = 0
fn = 0

for title, _ in similar_movies_familiy:
    if title in family_list:
        print(title)
        tp += 1
    else:
        fp += 1

rel = len(family_list)
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Toy Story
Aladdin
New York Minute
Big
Finding Nemo
Nightmare Before Christmas, The
Nightmare Before Christmas, The
Princess Bride, The
Pokemon: Mewtwo Returns
Coraline
Beauty and the Beast
Precision: 0.03691275167785235
Recall: 0.2619047619047619


# >>> 출력해보면 거의 모든 영화제목에 코사인 유사도 점수가 0.7 이상.. 의미가 없다
# >>> 그래서 그냥 top 300개가 아니라 다 출력하게 해봄

In [ ]:
def preprocess_text(text):
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # 특수 문자와 숫자 제거
        text = text.lower()  # 소문자로 변환
        text = re.sub(r'\s+', ' ', text).strip()  # 불필요한 공백 제거
        return text

def bert_embedding(text):
        inputs = bert_model_uiji.tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        outputs = bert_model_uiji.model(**inputs)
        sentence_embedding = outputs.last_hidden_state[:, 0, :].detach().numpy()
        return sentence_embedding

def generate_embeddings(scripts):
        return [bert_model_uiji.bert_embedding(script) for script in scripts]

def find_similar_movies(query): ###### 임의로
        query = bert_model_uiji.preprocess_text(query)  # Ensure query is preprocessed
        query_embedding = bert_model_uiji.bert_embedding(query)
        #print(query_embedding)
        similarities = [cosine_similarity(query_embedding, script_embedding).flatten()[0] for script_embedding in bert_model_uiji.script_embeddings]
        top_indices = np.argsort(similarities)[::-1]
        similar_movies = [(bert_model_uiji.title_list[i], similarities[i]) for i in top_indices]
        return similar_movies

In [ ]:
love_all = find_similar_movies("love")

In [ ]:
len(love_all[1:]) # 결과

1086

In [ ]:
len(script_list[1:]) # 데이터셋

1086

In [ ]:
tp = 0
fp = 0
fn = 0

for title, _ in love_all[1:]:
    if title in romance_list:
        print(title)
        tp += 1
    else:
        fp += 1

rel = len(romance_list)
fn = rel - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print("Precision:", precision)
print("Recall:", recall)

Punch-Drunk Love
Saint, The
Almost Famous
Meet Joe Black
Broadcast News
True Romance
Eternal Sunshine of the Spotless Mind
Ninotchka
High Fidelity
Wild At Heart
Fabulous Baker Boys, The
Crazy, Stupid, Love
Shakespeare in Love
Last of the Mohicans, The
Twilight
Burlesque
Crash
American President, The
Jane Eyre
From Here to Eternity
It's Complicated
Silver Linings Playbook
Juno
American Werewolf in London
Spanglish
Airplane
Pride and Prejudice
Bruce Almighty
Sessions, The
Larry Crowne
Midnight in Paris
Twilight: New Moon
Rocky
It Happened One Night
It Happened One Night
Monte Carlo
Crazylove
Her
Les Miserables
Zerophilia
Three Musketeers, The
His Girl Friday
Next Three Days, The
Yes Man
Cooler, The
Princess Bride, The
As Good As It Gets
Terminator
Up in the Air
Shipping News, The
Reader, The
Things My Father Never Taught Me, The
Tin Cup
Curious Case of Benjamin Button, The
Natural Born Killers
Mrs. Brown
Platinum Blonde
Ghost
Proposal, The
Youth in Revolt
Perks of Being a Wallflower, The

# truncated_svd도 이거처럼 다 그냥 유사도 높게나와서 거의 모든 영화를 출력할테고.. (0.25이상이지만) 그래서 recall은 높아질 수 밖에 없었던 것으로 강한 추측. 하이튼 그거 확인해보기. recall만 너무 높으면 품질 안좋은거임

# 시도해보면 좋을 것
1. truncated 출력 개수 top n 개만해서 "정확도" 봐보기
2. fine-tuning
3. 데이터 셋 줄이기
4. 영어가 아닌애들 없애기 전처리

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=1159d7fa2b3c74395db1784c8cd20286cf2a74ac5f44c845a0ebc288f3c16eb2
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import csv
import re
from langdetect import detect
csv.field_size_limit(100000000)

def is_english_word(word):
    try:
        return detect(word) == 'en'
    except:
        return False

def preprocess_script(script):
    words = script.split()
    processed_words = []

    for word in words:
        word = word.lower()  # 소문자로 변환
        if not is_english_word(word):  # 영어가 아닌 단어 제거
            continue
        processed_words.append(word)

    return ' '.join(processed_words)

def preprocess_csv_2(input_file, output_file, min_length=15):
    with open(input_file, 'r', encoding='utf-8') as f:
        rdr = csv.reader(f)
        rows = []

        for idx, line in enumerate(rdr):
            if len(line) <= 1:
                continue  # Skip rows with insufficient columns
            script = line[1]
            script = preprocess_script(script)

            rows.append([line[0], script])

    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['title', 'script'])  # Add header row
        writer.writerows(rows)

In [ ]:
pre_2 = '/content/uiji_preprocessd_scripts_2.csv'
pre_22 = '/content/uiji_preprocessd_scripts_22.csv'
preprocess_csv_2(pre_2, pre_22, min_length=15)